### Download and Prepare the Dog Cats Dataset

In [1]:
#!/bin/bash
!curl -L -o cats-and-dogs-mini-dataset.zip\
  https://www.kaggle.com/api/v1/datasets/download/aleemaparakatta/cats-and-dogs-mini-dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 21.8M  100 21.8M    0     0  48.4M      0 --:--:-- --:--:-- --:--:-- 48.4M


### Creating Dataset Structure for Training and Inferencing

In [6]:
!mkdir data
!unzip cats-and-dogs-mini-dataset.zip -d dataset

Archive:  cats-and-dogs-mini-dataset.zip
  inflating: dataset/cats_set/cat.4001.jpg  
  inflating: dataset/cats_set/cat.4002.jpg  
  inflating: dataset/cats_set/cat.4003.jpg  
  inflating: dataset/cats_set/cat.4004.jpg  
  inflating: dataset/cats_set/cat.4005.jpg  
  inflating: dataset/cats_set/cat.4006.jpg  
  inflating: dataset/cats_set/cat.4007.jpg  
  inflating: dataset/cats_set/cat.4008.jpg  
  inflating: dataset/cats_set/cat.4009.jpg  
  inflating: dataset/cats_set/cat.4010.jpg  
  inflating: dataset/cats_set/cat.4011.jpg  
  inflating: dataset/cats_set/cat.4012.jpg  
  inflating: dataset/cats_set/cat.4013.jpg  
  inflating: dataset/cats_set/cat.4014.jpg  
  inflating: dataset/cats_set/cat.4015.jpg  
  inflating: dataset/cats_set/cat.4016.jpg  
  inflating: dataset/cats_set/cat.4017.jpg  
  inflating: dataset/cats_set/cat.4018.jpg  
  inflating: dataset/cats_set/cat.4019.jpg  
  inflating: dataset/cats_set/cat.4020.jpg  
  inflating: dataset/cats_set/cat.4021.jpg  
  inflating: d

In [7]:
import os

# Create the 'data' directory if it doesn't exist
data_dir = 'data'
os.makedirs(data_dir, exist_ok=True)

# Create the 'train' and 'test' subdirectories within 'data'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [8]:
import os

# Define the source and destination paths
source_cat = 'dataset/cats_set'
dest_cat = 'dataset/cat'
source_dog = 'dataset/dogs_set'
dest_dog = 'dataset/dog'

# Rename the directories if they exist
if os.path.exists(source_cat):
  os.rename(source_cat, dest_cat)

if os.path.exists(source_dog):
  os.rename(source_dog, dest_dog)


### Creating the TRAIN TEST SPLIT within the DATA directory folders

In [9]:
import os
import random
import shutil

def train_test_split_folder(source_folder, train_folder, test_folder, split_ratio=0.8):
    """
    Splits a folder of images into training and testing sets.

    Args:
        source_folder: Path to the source folder containing subfolders for each class.
        train_folder: Path to the folder where the training set will be saved.
        test_folder: Path to the folder where the testing set will be saved.
        split_ratio: The ratio of images to include in the training set (default is 0.8).
    """

    if not os.path.exists(train_folder):
        os.makedirs(train_folder)
    if not os.path.exists(test_folder):
        os.makedirs(test_folder)

    for class_name in os.listdir(source_folder):
        class_source_path = os.path.join(source_folder, class_name)

        if os.path.isdir(class_source_path):  # Check if it is a directory
            train_class_path = os.path.join(train_folder, class_name)
            test_class_path = os.path.join(test_folder, class_name)

            if not os.path.exists(train_class_path):
                os.makedirs(train_class_path)
            if not os.path.exists(test_class_path):
                os.makedirs(test_class_path)

            images = [f for f in os.listdir(class_source_path) if os.path.isfile(os.path.join(class_source_path, f))]
            random.shuffle(images)
            split_index = int(len(images) * split_ratio)
            train_images = images[:split_index]
            test_images = images[split_index:]

            for image in train_images:
                source_path = os.path.join(class_source_path, image)
                destination_path = os.path.join(train_class_path, image)
                shutil.copy(source_path, destination_path)

            for image in test_images:
                source_path = os.path.join(class_source_path, image)
                destination_path = os.path.join(test_class_path, image)
                shutil.copy(source_path, destination_path)


# Example usage (assuming you have your data organized in a 'data/train' folder):
train_test_split_folder("dataset", "data/train", "data/test")

### Counting total Images in Folders

In [10]:
import os

def count_images_per_folder(root_folder):
  """
  Counts the number of images in each subfolder of a given root folder.

  Args:
    root_folder: The path to the root folder.

  Returns:
    A dictionary where keys are folder paths and values are the number of images in each folder.
  """

  image_counts = {}
  for dirpath, dirnames, filenames in os.walk(root_folder):
    image_count = 0
    for filename in filenames:
      if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        image_count += 1
    if image_count > 0 :
      image_counts[dirpath] = image_count
  return image_counts

# Example usage
image_counts = count_images_per_folder("data")
for folder, count in image_counts.items():
    print(f"Folder: {folder}, Number of images: {count}")


Folder: data/test/dog, Number of images: 100
Folder: data/test/cat, Number of images: 100
Folder: data/train/dog, Number of images: 400
Folder: data/train/cat, Number of images: 400


In [11]:
!sudo apt install tree -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tree is already the newest version (2.1.1-2ubuntu3.24.04.2).
0 upgraded, 0 newly installed, 0 to remove and 26 not upgraded.


### Directory Structure to start Training

In [12]:
!tree -d data


data
├── test
│   ├── cat
│   └── dog
└── train
    ├── cat
    └── dog

7 directories


### Necesary Imports

In [13]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

2026-01-12 10:02:25.777101: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


### Set paths to train and test folders

In [14]:
# Set paths to train and test folders
train_dir = "data/train"
test_dir = "data/test"
image_size = (128, 128)
batch_size = 32

### Data Preparation

In [15]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


train_data = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

test_data = datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 640 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


### Build the CNN Model

In [16]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # binary classification
])
model.summary()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2026-01-12 10:02:28.452934: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

### Model Compilation & Training

In [17]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Training the Model
checkpoint_path = 'dog_cat_cnn_model.keras'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True)

model.fit(
    train_data,
    epochs=20,
    validation_data=test_data,
    callbacks=[checkpoint]
)

Epoch 1/20


20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 253ms/step - accuracy: 0.5250 - loss: 0.6923 - val_accuracy: 0.5000 - val_loss: 0.6766
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 229ms/step - accuracy: 0.5734 - loss: 0.6759 - val_accuracy: 0.6250 - val_loss: 0.6627
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 229ms/step - accuracy: 0.6344 - loss: 0.6558 - val_accuracy: 0.6250 - val_loss: 0.6495
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 197ms/step - accuracy: 0.6141 - loss: 0.6403 - val_accuracy: 0.6250 - val_loss: 0.6394
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 208ms/step - accuracy: 0.6922 - loss: 0.6110 - val_accuracy: 0.6750 - val_loss: 0.6296
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 213ms/step - accuracy: 0.7156 - loss: 0.5819 - val_accuracy: 0.6250 - val_loss: 0.6046
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 209ms/step - accuracy: 0.7312 - loss: 0.5581 - val_accuracy: 0.6750 - val_loss: 0.5904
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 208ms/step - accuracy: 0.7422 - loss: 0.5433 - val_accuracy: 0.7000 - val_

### Saving the Trained Model

In [18]:
# Save final model
model.save('dog_cat_final_model1.keras')

### Loading the Trained Model

In [19]:
from tensorflow.keras.models import load_model

# Load the best saved model
model = load_model('dog_cat_final_model1.keras')

### Model Evaluation

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set parameters
val_dir = test_dir  # same directory used for training
image_size = (128, 128)
batch_size = 32

# Recreate the validation generator
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Evaluate model accuracy
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Found 50 images belonging to 2 classes.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6000 - loss: 0.6925 
Validation Accuracy: 60.00%


### Model Inferencing

In [21]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]

    if prediction > 0.5:
        print("Predicted: Dog")
    else:
        print("Predicted: Cat")

In [22]:
predict_image("data/train/cat/cat.4003.jpg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Predicted: Cat


In [25]:
predict_image("data/test/dog/dog.4095.jpg", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted: Dog
